---
layout: post
title: python goertzel DTMF 检测算法
category: dsp
tags: [dsp, ai, python]
---

In [21]:
%matplotlib notebook
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from scipy.fftpack import fft

sample_rate = 8000

N = 256

f_step = sample_rate / N

t = np.arange(0, 1, 1/sample_rate)

dtmf_freqs = np.array([697, 770, 852, 941, 1209, 1336, 1477, 1633])


f1 = np.sin(2 * np.pi * f_step * 20 * t)
f2 = np.sin(2 * np.pi * f_step * 40 * t)

x = f1 + f2

X = fft(x[:N], n = N)

print('===== FFT Result =========')
print('index 20 : ', X[20], np.abs(X[20]))
print('index 40 : ', X[40], np.abs(X[40]))

def goertzel(x, detect_freq, sample_rate):
    N = len(x)
    f_step = sample_rate / N
    k = int(detect_freq / f_step)
    
    v_cos = np.cos(2 * np.pi * k / N)
    v_sin = np.sin(2 * np.pi * k / N)
    coef = 2 * v_cos
    
    g1, g2 = 0.0, 0.0
    for i in range(N):
        _y = x[i] + coef * g1 - g2
        g2 = g1
        g1 = _y

    o_real = g1 - v_cos * g2
    o_image = v_sin * g2
    o_real = v_cos * g1 - g2
    o_image = v_sin * g1
    amp = g1 ** 2 + g2 ** 2 - g2 * g1 * coef
    return (o_real + 1j * o_image, amp)


print('\n===== Goertzel Result =========')
x20, amp = goertzel(x[:N], f_step * 20, sample_rate)
print('goertzel 20 : ', x20, np.abs(x20))

x40, amp = goertzel(x[:N], f_step * 40, sample_rate)
print('goertzel 40 : ', x40, np.abs(x40))



===== FFT Result =========
index 20 :  (-8.346996846337895e-13-127.99999999999994j) 127.99999999999994
index 40 :  (-1.7659087001539012e-12-128j) 128.0

===== Goertzel Result =========
goertzel 20 :  (-1.5347723092418164e-12-128.00000000000003j) 128.00000000000003
goertzel 40 :  (-2.9274360713316128e-12-128j) 128.0


# 参考

* [Wiki : Goertzel_algorithm](https://en.wikipedia.org/wiki/Goertzel_algorithm)
* [python 实现 goertzel 算法代码](https://gist.github.com/sebpiq/4128537)